In [1]:
import numpy as np
from datetime import datetime

from kadlu import source_map
from kadlu.geospatial.ocean import Ocean

The ocean module is an abstraction of the loading and interpolation functions to quickly load and interpolate many different environmental variables at once. By default, the ocean will create interpolators using null data:

In [2]:
# ocean boundaries: gulf of st lawrence (top 100m from the surface)
bounds = dict(
        start=datetime(2015, 1, 9), end=datetime(2015, 1, 9, 12),
        south=44,                   west=-68.5, 
        north=51,                   east=-56.5, 
        top=0,                      bottom=100
    )

In [3]:
# ocean initialized without any environmental data will return null (zeroes)
o = Ocean(**bounds)                              # instantiate interpolators using null values
o.bathy(lat=[47.5, 48.1], lon=[-64, -62.5])      # query bathymetric interpolator for values at given coordinates

array([0., 0.])

To prepare an interpolation using Kadlu's automated data loading, specify one of the data sources listed in the source_map:

In [4]:
o = Ocean(load_salinity='hycom', **bounds)                     # instantiate interpolator with HYCOM salinity data
o.salinity(lat=[47.5, 48.1], lon=[-64, -62.5], depth=[0, 10])  # query interpolator for values at given coordinates

array([30., 31.])

In [5]:
o.waveheight(lat=[47.5, 48.1], lon=[-64, -62.5]) # query waveheight interpolator: values remain null

array([0., 0.])

Kadlu can check for missing data and prepare interpolators for many variables at the same time

In [6]:
# source strings passed as load arguments tells the ocean module where to source the data
sources = dict(
        load_bathymetry=0, # dont load bathymetry 
        load_temp='hycom',
        load_salinity='hycom',
        load_wavedir='era5',
        load_waveheight='era5',
        load_waveperiod='era5',
        load_wavedirection='era5',
        load_windspeed='era5'
    )

o = Ocean(**sources, **bounds)

In [7]:
o.waveheight(lat=[47.5, 48.1], lon=[-64, -62.5])  # query waveheight interpolator

array([1.80722759, 2.43859011])

Kadlu also supports ocean interpolation using arbitrary environmental data. An ocean can be initialized by passing a float or array of floats to the load_variable keyword argument. Arrays must be ordered by [values, lat, lon] for 2D data, or [values, lat, lon, depth] for 3D data.

When a float value is used, a uniform "interpolation" of that value will be returned for every coordinate location

In [8]:
bathy_arr = np.array((
        np.random.randint(0, 500, 100),                            # values
        np.random.randint(bounds['south'], bounds['north'], 100),  # latitudes
        np.random.randint(bounds['west'],  bounds['east'], 100)    # longitudes
    ))
temp_float = 10
salinity_float = 35

o = Ocean(load_bathymetry=bathy_arr, load_temp=temp_float, load_salinity=salinity_float, **bounds)

In [9]:
print('bathymetry:\t',  o.bathy(   lat=[47.5, 48.1], lon=[-64, -62.5]))
print('temperature:\t', o.temp(    lat=[47.5, 48.1], lon=[-64, -62.5], depth=[0, 50]))
print('salinity:\t',    o.salinity(lat=[47.5, 48.1], lon=[-64, -62.5], depth=[0, 50]))

bathymetry:	 [398.16666667 346.1       ]
temperature:	 [10. 10.]
salinity:	 [35. 35.]


for a list of possible strings accepted as input for the load_variable arguments, choose one of the sources listed in the source_map that supports that variable:

In [10]:
print(source_map)


    CHS   (Canadian Hydrography Service)
          load_bathymetry:          bathymetric data in Canada's waterways. variable resolution 

    ERA5  (Global environmental dataset from Copernicus Climate Data Store)
          load_windwaveswellheight: combined height of wind, waves, and swell. metres
          load_wavedirection:       mean wave direction, degrees
          load_waveperiod:          mean wave period, seconds
          load_wind_uv:             wind speed computed as sqrt(u^2 + v^2) / 2, where u, v are direction vectors
          load_wind_u:              wind speed coordinate U-vector, m/s
          load_wind_v:              wind speed coordinate V-vector, m/s 

    HYCOM (Hybrid Coordinate Ocean Model)
          load_salinity:            g/kg salt in water
          load_temp:                degrees celsius
          load_water_uv:            ocean current computed as sqrt(u^2 + v^2) / 2, where u, v are direction vectors
          load_water_u:             ocean curre